In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))
    ])

In [3]:
batch_size = 4

train_set = torchvision.datasets.MNIST(root='./data', train =True, download =True,
                                      transform = transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size,
                                          shuffle = True,
                                          num_workers = 2)
test_set = torchvision.datasets.MNIST(root = './data', train =False,
                                     download = True, transform = transform)

test_loader = torch.utils.data.DataLoader(test_set, batch_size = batch_size,
                                         shuffle = False, num_workers = 2)

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # # our first conv layer
        
        # self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3, 3), stride = 1, padding =1)
        
        # #max pool layer
        # self.pool = nn.MaxPool2d(kernel_size = (2, 2), stride = (2,2 ))
        
        # # our second conv layer
        # self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3, 3), stride =1, padding =1)
        
        # # our fully connected layer
        
        # self.fc1 = nn.Linear(16*7*7, 16) # flatten your tensor
        
        # # our second fully connected layer
        # self.fc2 = nn.Linear(16, 10)

        self.fc = nn.Linear(784, 100)
        self.fcl = nn.Linear(100, 10)

        

    def forward(self, X):
        
        X = X.reshape(X.shape[0], -1)
        X = self.fc(X)
        X = self.fcl(X)
        
        return X

In [15]:
net=CNN()
print(net)

CNN(
  (fc): Linear(in_features=784, out_features=100, bias=True)
  (fcl): Linear(in_features=100, out_features=10, bias=True)
)


In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)


In [17]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.01)

In [18]:

train_loss = []
train_accuracy = []

In [19]:
for epoch in range(2):
    running_loss = 0.0
    running_acc = 0.0
    total = 0.0
    
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        
        images =images.to(device)
        labels = labels.to(device)
        
        outputs = net(images).to(device)
        
        optimizer.zero_grad()
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _,predicted = torch.max(outputs, 1)
        total += labels.size(0)
        
        running_acc += (predicted == labels).sum().item()
        running_loss += loss.item()
        
    train_loss = running_loss / total
    train_acc = 100*running_acc / total
    
    print('Epoch: [%d] loss: %.3f' % (epoch + 1, running_loss /(len(train_loader))))
    print('Epoch: ', epoch + 1, 'Training accuracy %d %%' %(train_acc))
    


RuntimeError: mat1 and mat2 shapes cannot be multiplied (112x28 and 784x100)

In [13]:
test_acc = 0.0
total = 0.0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images).to(device)
        
        _,predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        test_acc += (predicted == labels).sum().item()
        
print(f'Accuracy of the network on test images: {100 * test_acc // total}%')

Accuracy of the network on test images: 88.0%
